In [1]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dropout, Input, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [2]:
batch_size = 8;

Train_DataSet_Path = r'D:\Paul\CHS_Proiect\Datasets\Eyes_DataSet\Train_Dataset'

train_data_generator = ImageDataGenerator(rotation_range=0.2, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,zoom_range=0.2, rescale=1./255, validation_split=0.2)
train_data = train_data_generator.flow_from_directory(Train_DataSet_Path,
                                               target_size=(80, 80), batch_size=batch_size, class_mode='categorical',
                                               subset='training')
validation_data = train_data_generator.flow_from_directory(Train_DataSet_Path,
                                                    target_size=(80, 80), batch_size=batch_size, class_mode='categorical',
                                                    subset='validation')

Found 54334 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [3]:
Test_DataSet_Path = r'D:\Paul\CHS_Proiect\Datasets\Eyes_DataSet\Test_Dataset'

test_data_generator = ImageDataGenerator(rescale=1./255)
test_data = test_data_generator.flow_from_directory(Test_DataSet_Path,
                                             target_size=(80, 80), batch_size=batch_size, class_mode='categorical')

Found 16981 images belonging to 2 classes.


In [4]:
base_model = InceptionV3(include_top=False, weights= "imagenet", input_tensor=Input(shape=(80, 80, 3)))
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(64, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation='softmax')(head_model)

final_model =  Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
    layer.trainable = False


In [5]:
checkpoint = ModelCheckpoint(r'D:\Paul\CHS_Proiect\Models\model.h5', monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)
callbacks = [checkpoint, earlystop, learning_rate]



In [6]:
final_model.compile(loss='categorical_crossentropy', optimizer='Adam',  metrics=['accuracy'])
final_model.fit(train_data, steps_per_epoch=train_data.samples//batch_size, validation_data=validation_data, validation_steps=validation_data.samples//batch_size, callbacks=callbacks, epochs=5)  #8 is the batch_size

Epoch 1/5
6791/6791 [==============================] - ETA: 0s - loss: 0.1960 - accuracy: 0.9247
Epoch 1: val_loss improved from inf to 0.27306, saving model to D:\Paul\CHS_Proiect\Models\model.h5
6791/6791 [==============================] - 1970s 289ms/step - loss: 0.1960 - accuracy: 0.9247 - val_loss: 0.2731 - val_accuracy: 0.8985 - lr: 0.0010
Epoch 2/5
6790/6791 [============================>.] - ETA: 0s - loss: 0.1691 - accuracy: 0.9357
Epoch 2: val_loss did not improve from 0.27306
6791/6791 [==============================] - 1411s 208ms/step - loss: 0.1691 - accuracy: 0.9358 - val_loss: 0.2733 - val_accuracy: 0.8950 - lr: 0.0010
Epoch 3/5
6791/6791 [==============================] - ETA: 0s - loss: 0.1583 - accuracy: 0.9404
Epoch 3: val_loss improved from 0.27306 to 0.25681, saving model to D:\Paul\CHS_Proiect\Models\model.h5
6791/6791 [==============================] - 432s 64ms/step - loss: 0.1583 - accuracy: 0.9404 - val_loss: 0.2568 - val_accuracy: 0.9053 - lr: 0.0010
Epoch 4

In [7]:
print(final_model.evaluate(train_data))
print(final_model.evaluate(validation_data))
print(final_model.evaluate(test_data))

6792/6792 [==============================] - 343s 51ms/step - loss: 0.1271 - accuracy: 0.9517
[0.12713490426540375, 0.9516876935958862]
1698/1698 [==============================] - 87s 51ms/step - loss: 0.2711 - accuracy: 0.8934
[0.27109211683273315, 0.8933961391448975]
2123/2123 [==============================] - 349s 165ms/step - loss: 0.2867 - accuracy: 0.9130
[0.2867225110530853, 0.9130204319953918]
